## Part 3

### Create map of neighborhoods in Toronto

In [27]:
import pandas as pd
import numpy as np
#!conda install -c conda-forge folium=0.5.0 --yes
import folium
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# import k-means from clustering stage
from sklearn.cluster import KMeans
import requests

### Use geopy library to get the latitude and longitude values of Toronto, Canada

In [16]:
nb_data = pd.read_csv('Toronto_Neighborhoods2.csv')
nb_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [17]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent='ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6534817, -79.3839347.


### Visualizing the group map


In [26]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, neighborhood, borough,postalcode in zip(nb_data['Latitude'],nb_data['Longitude'],nb_data['Neighborhood'],nb_data['Borough'],nb_data['PostalCode']):
    label = '{},{},{}'.format(neighborhood, borough,postalcode)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_Toronto) 
map_Toronto

### Explore and cluster the neighborhoods in Toronto

### Define Foursquare Credentials and Version

In [19]:
CLIENT_ID = 'BXZDLEBHVSDCTCOY3DUERFGQDXPUEVRDBECETJJJ0TR2C3WL' # your Foursquare ID
CLIENT_SECRET = '2VQ3CQF3KF245LL5WDW5HWZ0BREQUQDRRE3TA0GRHUP3NFBS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BXZDLEBHVSDCTCOY3DUERFGQDXPUEVRDBECETJJJ0TR2C3WL
CLIENT_SECRET:2VQ3CQF3KF245LL5WDW5HWZ0BREQUQDRRE3TA0GRHUP3NFBS


In [20]:
nb_data.loc[0,"Neighborhood"]

'Rouge,Malvern'

In [21]:
neighborhood_latitude = nb_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = nb_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = nb_data.loc[0, 'Neighborhood'] # neighborhood name

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

#Get the nearby Venues
def getNearbyVenues(names, latitudes, longitudes, postalcodes, radius=500):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng, postalcode in zip(names, latitudes, longitudes, postalcodes):
        #print(postalcode)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            postalcode,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'PostalCode',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(nb_data['Borough'].unique()),
        nb_data.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [24]:
print("Unique boroughs in the dataframe:")
print(nb_data['Borough'].unique())

Unique boroughs in the dataframe:
['Scarborough' 'North York' 'East York' 'East Toronto' 'Central Toronto'
 'Downtown Toronto' 'York' 'West Toronto' "Queen's Park" 'Mississauga'
 'Etobicoke']


In [28]:
Toronto_venues = getNearbyVenues(names=nb_data['Neighborhood'],
                                   latitudes=nb_data['Latitude'],
                                   longitudes=nb_data['Longitude'],
                                   postalcodes=nb_data['PostalCode']
                                  )

### check the size of the resulting dataframe

In [29]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2146, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,PostalCode,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,M1B,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,M1C,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,M1E,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,M1E,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,M1E,Sail Sushi,43.765951,-79.191275,Restaurant


### check how many venues were returned for each neighborhood

In [30]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,PostalCode,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100,100
Agincourt,4,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",8,8,8,8,8,8,8
"Alderwood,Long Branch",7,7,7,7,7,7,7
"Bathurst Manor,Downsview North,Wilson Heights",25,25,25,25,25,25,25
Bayview Village,4,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",23,23,23,23,23,23,23
Berczy Park,57,57,57,57,57,57,57


### find out how many unique categories can be curated from all the returned venues

In [31]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 267 uniques categories.


## Analyze Each Neighborhood

In [32]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
Toronto_onehot.shape

(2146, 267)

### group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [34]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.010000,0.00000,0.0,0.010000,0.00,0.000000,0.00,0.000000,0.0,0.01
1,Agincourt,0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.00000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.00000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.00000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00
4,"Alderwood,Long Branch",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.00000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00
5,"Bathurst Manor,Downsview North,Wilson Heights",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.00000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00
6,Bayview Village,0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.00000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00
7,"Bedford Park,Lawrence Manor East",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.043478,0.00000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00
8,Berczy Park,0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.00000,0.0,0.017544,0.00,0.000000,0.00,0.000000,0.0,0.00
9,"Birch Cliff,Cliffside West",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.00000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00


### confirm the new size

In [35]:
Toronto_grouped.shape

(99, 267)

### print each neighborhood along with the top 5 most common venues

In [37]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.05
2        Hotel  0.04
3          Bar  0.04
4          Gym  0.04


----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1               Skating Rink  0.25
2  Latin American Restaurant  0.25
3                     Lounge  0.25
4   Mediterranean Restaurant  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                       venue  freq
0                       Park   0.5
1                 Playground   0.5
2             Medical Center   0.0
3         Miscellaneous Shop   0.0
4  Middle Eastern Restaurant   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store  0.25
1  Fast Food Restaurant  0.12
2            Beer Store  0.12
3              Pharmacy  0.12
4           Pizza Place  0.12


----Alderwood,Long Branch----
            ve

                venue  freq
0    Department Store   0.2
1         Coffee Shop   0.2
2   Convenience Store   0.2
3          Hobby Shop   0.2
4  Chinese Restaurant   0.2


----East Toronto----
                      venue  freq
0                      Park   0.5
1         Convenience Store   0.5
2  Mediterranean Restaurant   0.0
3         Mobile Phone Shop   0.0
4        Miscellaneous Shop   0.0


----Emery,Humberlea----
                       venue  freq
0             Baseball Field   0.5
1               Food Service   0.5
2                Yoga Studio   0.0
3         Miscellaneous Shop   0.0
4  Middle Eastern Restaurant   0.0


----Fairview,Henry Farm,Oriole----
                  venue  freq
0        Clothing Store  0.08
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.06
3            Restaurant  0.05
4         Women's Store  0.05


----First Canadian Place,Underground city----
                 venue  freq
0          Coffee Shop  0.11
1                 Café  0.07
2                

                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.06
2         Cocktail Bar  0.05
3  American Restaurant  0.04
4             Beer Bar  0.04


----Stn A PO Boxes 25 The Esplanade----
                 venue  freq
0          Coffee Shop  0.09
1   Seafood Restaurant  0.04
2                  Pub  0.04
3                 Café  0.04
4  Japanese Restaurant  0.03


----Studio District----
         venue  freq
0         Café  0.10
1  Coffee Shop  0.08
2       Bakery  0.05
3    Gastropub  0.05
4      Brewery  0.05


----The Annex,North Midtown,Yorkville----
               venue  freq
0               Café  0.14
1     Sandwich Place  0.14
2        Coffee Shop  0.09
3           Pharmacy  0.05
4  Indian Restaurant  0.05


----The Beaches----
                      venue  freq
0                     Trail  0.25
1                       Pub  0.25
2         Health Food Store  0.25
3  Mediterranean Restaurant  0.00
4        Miscellaneous Shop  0.00


----The Beaches West,Ind

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### put that into a pandas dataframe

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Restaurant,Hotel,Bar,Gym,Clothing Store,Thai Restaurant,Concert Hall,American Restaurant
1,Agincourt,Skating Rink,Latin American Restaurant,Lounge,Breakfast Spot,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Fast Food Restaurant,Beer Store,Fried Chicken Joint,Pharmacy,Pizza Place,Sandwich Place,Distribution Center,Dessert Shop,Dim Sum Restaurant
4,"Alderwood,Long Branch",Pizza Place,Pub,Gym,Skating Rink,Sandwich Place,Coffee Shop,Drugstore,Donut Shop,Doner Restaurant,Dog Run


In [41]:
Toronto_grouped.shape[0]
Toronto_grouped.iloc[0, :]

Neighborhood                     Adelaide,King,Richmond
Yoga Studio                                           0
Accessories Store                                     0
Afghan Restaurant                                     0
Airport                                               0
Airport Food Court                                    0
Airport Gate                                          0
Airport Lounge                                        0
Airport Service                                       0
Airport Terminal                                      0
American Restaurant                                0.02
Antique Shop                                          0
Aquarium                                              0
Art Gallery                                        0.01
Art Museum                                         0.01
Arts & Crafts Store                                   0
Asian Restaurant                                   0.01
Athletics & Sports                              

## Cluster Neighborhoods

In [42]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [43]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = nb_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Women's Store,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,4.0,Bar,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Falafel Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0.0,Mexican Restaurant,Rental Car Location,Restaurant,Medical Center,Bank,Intersection,Electronics Store,Breakfast Spot,Dog Run,Distribution Center
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Bakery,Lounge,Bank,Hakka Restaurant,Athletics & Sports,Fried Chicken Joint,Gas Station,Caribbean Restaurant,Thai Restaurant,Women's Store


In [44]:
Toronto_merged['Cluster Labels'].unique()

array([ 0.,  4.,  2., nan,  1.,  3.])

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    #print (poi, cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    # handle when cluster is nan
    cluster_ind = -1
    if (np.isnan(cluster)):
        cluster_ind = 5
    else:
        cluster_ind = int(cluster)
        
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster_ind-1],
        fill=True,
        fill_color=rainbow[cluster_ind-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters